In [1]:
#SHORTCUTS
    #ShiftEnter run current cell and move to the next
    #CtrlEnter run current cell
    #Ctrl/ toggle commenting of line

In [2]:
#so that notebook fills screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

print("Done adjusting...")

Done adjusting...


In [4]:
#Welcome to the demo/tutorial for Joe Faraguna's Python pipeline for MS/phenotypic data analysis! 
#The code I've written for the pipeline is contained within 2 python files, classes.py and modules.py
    #classes contains information about the structure of the Experiments that you can create and handles adding replicates and calling analysis or plot functions on your data
    #modules contains more basic functions that are called by the functions within classes - classes shuttles most of the analysis to modules and then returns it to the user
#As a user, you will not be interacting with most of the code in either of these two files unless you decide to extend the program with additional functions
    
#As a user, the general workflow is:
    #1) LOAD DATA - specify where your csv files are that store MS data, specify cell lines names and time points
    #2) CREATE EXPERIMENT - relate this data to an Experiment, a data structure represented in classes.py
    #3) RUN ANALYSIS - call Experiment functions to generate plots and files

#This demo also includes:
    #4) TEMPLATE AND REFERENCE - copy this template into a new file for quick setup with your own data

#You need to import these every time you want to run the pipeline
import classes
import modules
#This is a visualization package for plotting graphs in python. It is also required.
import matplotlib.pyplot as plt

print("Done importing...")

Done importing...


In [5]:
#1) LOAD DATA

#Now that we've installed the pipeline and a plotting package, we have to load our data
    #I'm using Jacqueline's Gerritsen's MS data on EGFR tyrosine point-mutation mutants here
    #I'd recommend working through this demo completely with her data before formatting your own data for input so that you have a good understanding of how to use the pipeline!

#Here, we begin with MS data, although we can also load phenotypic data

#locations is a list of lists
    #each list stores the locations of all the .csv files for one experimental replicate
#in this case, we have 3 experimental replicates with 8 cell lines each: 2934, 2935, 2936, 2937, 2938, 3126, 3127, and 3138

#NOTE: each experimental replicate includes the cell line data in the same order
#NOTE: the pipeline assumes the last cell line, in this case 3138, is the reference cell line (wt, untreated, etc, in this case WT EGFR receptor), although this can be changed later
locations = [
	["demo/input/ms/2934_2nM_BR1.csv",#the first replicate
	"demo/input/ms/2935_2nM_BR1.csv",
	"demo/input/ms/2936_2nM_BR1.csv",
	"demo/input/ms/2937_2nM_BR1.csv",
	"demo/input/ms/2938_2nM_BR1.csv",
	"demo/input/ms/3126_2nM_BR1.csv",
	"demo/input/ms/3127_2nM_BR1.csv",
	"demo/input/ms/3138_2nM_BR1.csv"],
	["demo/input/ms/2934_2nM_BR2.csv",#the second replicate
	"demo/input/ms/2935_2nM_BR2.csv",
	"demo/input/ms/2936_2nM_BR2.csv",
	"demo/input/ms/2937_2nM_BR2.csv",
	"demo/input/ms/2938_2nM_BR2.csv",
	"demo/input/ms/3126_2nM_BR2.csv",
	"demo/input/ms/3127_2nM_BR2.csv",
	"demo/input/ms/3138_2nM_BR2.csv"],
	["demo/input/ms/2934_2nM_BR3.csv",#the third replicate
	"demo/input/ms/2935_2nM_BR3.csv",
	"demo/input/ms/2936_2nM_BR3.csv",
	"demo/input/ms/2937_2nM_BR3.csv",
	"demo/input/ms/2938_2nM_BR3.csv",
	"demo/input/ms/3126_2nM_BR3.csv",
	"demo/input/ms/3127_2nM_BR3.csv",
	"demo/input/ms/3138_2nM_BR3.csv"]]
#These file locations are RELATIVE locations, meaning they supply a path relative to the file we're running
    #You could also supply an ABSOLUTE file location like 'D:/Dropbox (MIT)/pipeline/demo/input/ms/2934_2nM_BR1.csv',
        #but then if you change the location of the pipeline folder or use a different computer, you have to change the locations.

#Open pipeline/demo/input/ms/2934_2nM_BR1.csv to see the structure of these files
    #The first two columns are the peptide sequences and protein descriptions from MS analysis
        #Keep the full protein description (OX = ... GN = ...) because the pipeline will sometimes display GNs for short
        #The pipeline works if you supply Master Protein Descriptions or Protein Descriptions with multiple protein guesses
            #The first protein guess will be separated and used for some labelling, but the other guesses will always be accessible in saved plots
    #The next n columns are peptide abundance measurements that have been filtered for high confidence
        #and normalized to the SUP channel so that different runs can be compared.
    #Including the header row with titles is required, although the names of the columns will be written over by the pipeline
        #However, the n columns of abundance measurements should be in time order
        #So we could have columns that say 'pep  pro  0  30  1  2  5' or anything else as long as the two description columns are ordered correctly and the measurements are in time order

#We can also load technical replicates, which will be combined differently than experimental ones
technicalReplicate = ["demo/input/ms/2934_2nM_BR2TR.csv",
	"demo/input/ms/2935_2nM_BR2TR.csv",
	"demo/input/ms/2936_2nM_BR2TR.csv",
	"demo/input/ms/2937_2nM_BR2TR.csv",
	"demo/input/ms/2938_2nM_BR2TR.csv",
	"demo/input/ms/3126_2nM_BR2TR.csv",
	"demo/input/ms/3127_2nM_BR2TR.csv",
	"demo/input/ms/3138_2nM_BR2TR.csv"]
#Open pipeline/demo/input/ms/2934_2nM_BR2TR.csv to see the structure of these files
    #These are structured the same as MS replicate data

#We can also load phenotypic data replicates.
phenotypicMeasurement = ['demo/input/ph/PhenoClass 1.csv',
	'demo/input/ph/PhenoClass 3.csv',
	'demo/input/ph/PhenoClass 4.csv']
#Open pipeline/demo/input/ph/PhenoClass1.csv to see the structure of these files
    #The two columns are Cell Line and a numerical replicate identifier column (e.g. 1, 2...)
    #Each phenotypic replicate should have one measurement per cell line
    #Including the hreader row with Cell Line and a numerical replicate identifier is required

#Finally, we have to include information about the names of the cell lines and the time points we gather MS data at
cell_lines = ['2934','2935','2936','2937','2938','3126','3127','3138'] #strings
time_points = [0, 30, 1, 2, 5] #ints
second_time_points = [0, 30, 60, 120, 300] #ints

print("Done listing data...")

Done listing data...


In [6]:
#2) CREATE EXPERIMENT

#Now that we've listed the file locations, cell line names, and time points, we can create the Experiment.
#The pipeline is built around Experiments, which is a structure for organizing MS and phenotypic data about related cell lines and for analyzing and plotting the data.
    #More on this later.
    
#creates experiment for all 3 replicates
exp = classes.Experiment(locations, cell_lines, time_points, second_time_points, names = ["BR1", "BR2", "BR3"], fileLocation = 'demo/output/')

#creates experiment for BR1 and BR2, not BR3
exp2 = classes.Experiment([locations[0],locations[1]], cell_lines, time_points, second_time_points, names = ["BR1", "BR2"], fileLocation = 'demo/output/')

#We also specified a fileLocation for the Experiment: this is the default location for saving plots and Excel files that we generate, although we can override it whenever we call a plotting function.

#We can call print() on the experiments to check that everything loaded correctly. Doing this will display information about each MS data replicate within the Experiment.
    #This includes the number of peptides for each cell line as well as the number of peptides that overlap between all cell lines for that given replicate.
print("~~~EXPERIMENT 1~~~")
print(exp)
print("\n~~~EXPERIMENT 2~~~")
print(exp2)

#Notice that exp2 is just exp except without the 3rd replicate, BR3

~~~EXPERIMENT 1~~~
BR1 MS REPLICATE
Cell Lines: '2934', '2935', '2936', '2937', '2938', '3126', '3127', '3138'
Size: 458, 458, 505, 505, 337, 337, 399, 399
Intersection Size: 196

BR2 MS REPLICATE
Cell Lines: '2934', '2935', '2936', '2937', '2938', '3126', '3127', '3138'
Size: 524, 355, 531, 345, 524, 355, 531, 345
Intersection Size: 188

BR3 MS REPLICATE
Cell Lines: '2934', '2935', '2936', '2937', '2938', '3126', '3127', '3138'
Size: 210, 370, 279, 433, 433, 279, 370, 210
Intersection Size: 112


~~~EXPERIMENT 2~~~
BR1 MS REPLICATE
Cell Lines: '2934', '2935', '2936', '2937', '2938', '3126', '3127', '3138'
Size: 458, 458, 505, 505, 337, 337, 399, 399
Intersection Size: 196

BR2 MS REPLICATE
Cell Lines: '2934', '2935', '2936', '2937', '2938', '3126', '3127', '3138'
Size: 524, 355, 531, 345, 524, 355, 531, 345
Intersection Size: 188



In [ ]:
#Let's also add the technical replicate to exp while we're at it
#The technical replicate was made along with BR2, so let's add it to BR2
    #Since Python is 0-indexed, we say i=1 because our replicates are [BR1, BR2, BR3]
exp.addTechnicalReplicate(technicalReplicate,i = 1)
#Notice that BR2 now has a lot more peptide data than before, although BR1 and BR3 are unchanged
    #When the program adds a technical replicate, it takes the union of all of the peptide measurements between the technical replicate and the other replicate data, taking the mean for repeated measurements.
print(exp)

In [ ]:
#One of the useful ways we can figure out more about the pipeline is by looking at the different structures and functions stored in it.
#Two useful tools are help() and dir(), which can be called on any variable we create, although they're most useful for Experiments and their functions

#help(exp) will list all the functions associated with exp and will include additional information about the different parameters you can set
    #help() is kind of overwhelming if you call it on the full exp:
help(exp)

In [ ]:
#dir(exp) will list all the functions and structures associated with exp
    #dir() is better for getting a general idea of Experiments, although it doesn't specify whether something is a function or a structure
    #it's also best to ignore the functions of the form __blahblahblah__: you won't be calling them as an end user (they are private)
dir(exp)

In [ ]:
#If we glance at dir, we can see that exp has a structure associated with it called experimentalReplicates. Let's try to call it:
print(exp.experimentalReplicates)
#it seems like a list of ExperimentalReplicate structures, in this case BR1, BR2, and BR3, although the printout is confusing and doesn't really help us do anything

In [7]:
#We can index the list to get just one instead:
print(exp.experimentalReplicates[0])
#Much better... If we compare this to print(exp), we can see that this is indeed the 1st replicate we loaded into the pipeline
    #Python is 0-indexed

#Each Experiment like exp stores MS data as separated ExperimentalReplicates

BR1 MS REPLICATE
Cell Lines: '2934', '2935', '2936', '2937', '2938', '3126', '3127', '3138'
Size: 458, 458, 505, 505, 337, 337, 399, 399
Intersection Size: 196



In [ ]:
#To make things easier, instead of calling exp.experimentalReplicates[0] whenever we want the 1st replicate of an Experiment, we can just index the Experiment directly:
print(exp[0])
#Try to print out the 3rd replicate:


In [ ]:
#We can also call help() and dir() on these ExperimentalReplicates directly:
dir(exp[0])

In [ ]:
#or on specific functions we see
help(exp.heatmap)

In [ ]:
#At the core of each ExperimentalReplicate is just the MS data we loaded in the first place
    #This is stored in cellData
    #Each cell line's MS data is in a separate data structure inside of cellData
#Let's print the 1st replicate's MS data
print(exp[0].cellData)

In [8]:
#each replicate's cellData is a list of data structures that each store one cell line's MS data
#here we call BR1's data on the first cell line only, 2934
#notice the blank column called Overflow Protein Descriptions
    #if you use files with Protein Descriptions instead of Master Protein Descriptions, the first protein hit will be included as Master and all the others will be in Overflow
print(exp[0].cellData[0])

                        peptide-phosphosite  \
0    AAEAAASAYYNPGNPHNVYMPTSQPPPPPYYPPEDK-1   
1                         AAEDNPYWVSPAYSK-1   
2        AAPSHCSLTPCGEDYEEEGLELAPETDGRPPR-1   
3                      AAVPSGASTGIYEALELR-1   
4                         ACLDDSYASGEGLKR-1   
..                                      ...   
453       YSSDPTGALTEDSIDDTFLPVPEYINQSVPK-1   
454       YSSDPTGALTEDSIDDTFLPVPEYINQSVPK-2   
455                        YVDSEGHLYTVPIR-1   
456                     YVLCPSTAPGPTLSADR-1   
457                       YYEPYYAAGPSYGGR-1   

                           Master Protein Descriptions  \
0    WW domain-binding protein 2 OS=Mus musculus OX...   
1    WD repeat-containing protein 70 OS=Mus musculu...   
2    Rho guanine nucleotide exchange factor 40 OS=M...   
3    Alpha-enolase OS=Mus musculus OX=10090 GN=Eno1...   
4    IQ motif and SEC7 domain-containing protein 1 ...   
..                                                 ...   
453  Epidermal growth factor 

In [ ]:
#Let's try actually producing a plot now, maybe a heatmap plot as above
exp.heatmap()
#However, before we can generate heatmap plots, we have to merge the separate replicates to create overall peptide abundance levels for the entire Experiment

In [ ]:
#Combining replicates is easy!
exp.combineReplicates()
print(exp)
#Now when we print(exp), we can see the overall values for each cell line and the intersection of all of these overall cell line values
    #When we combine replicates, we average all the available replicate abundances for each peptide: these values are used for plots like heatmap

In [ ]:
#You may notice that there is an 'n' and a 'std dev' listed with the combined replicates.
#These are cutoffs the user can enforce to try to eliminate low-confidence peptide measurements
    #For example, we might want to only keep peptides that show up in at least 2 replicates (>=2) and that have a small standard deviation across the replicates (<=0.2, for example)
#If you don't specify cutoffs, the pipeline selects all peptides, even if they only show up once and even if they have a very high standard deviation across replicates
    
#Try to use help() to figure out how to combine the replicates with different cutoffs! You can specify both cutoffs, neither, or just one
    #If you use n=2 and std=0.2 as the cutoffs, you should get an experimental intersection size of just 15
    #If you use n=3 and std=0 as cutoffs, you should get a warning that there are no peptides left (no peptide exists in all 3 replicates and has a std dev of 0)
    #If you use n=1, you should get an experimental intersection size of 421


In [ ]:
#3A) RUN ANALYSIS - PLOTS

#Now let's plot our heatmap!
#call the function
exp.heatmap()
#then display the plot
#NOTE: this is not strictly necessary when working with Jupyter Notebooks (.ipynb), but it is required when running Python (.py) files
plt.show()

In [ ]:
#if we look at the heatmap function, we can see that there are many different parameters we can set
help(exp.heatmap)
#the most useful ones are display, saveFig, saveFile, fileLocation, and normalization

In [ ]:
#we can save our heatmap to the default file location for our experiment instead of displaying it
    #we set this in cell 5 to be demo/output/
exp.heatmap(display=False, saveFig=True)

In [ ]:
#we can also save our heatmap as a .csv file so we can check out the identities of the peptides that seem interesting
exp.heatmap(display=False, saveFile=True)

In [ ]:
#a critical part of the pipeline is to be able to adjust the normalization scheme when plotting or analyzing
exp.heatmap() #default is 'refbasal' (normalize to reference cell line's basal/first time point)
exp.heatmap(normalization='ownbasal') #normalize each cell line to its own basal/first time point
exp.heatmap(normalization='reftime') #normalize each time point to that reference time point

In [ ]:
#But what if we want to change the reference cell line?
exp.setReference('2935')
exp.heatmap(normalization = 'reftime')

In [ ]:
#We can also target individual replicates directly
exp[0].heatmap(normalization = 'reftime')
#Notice there are a lot fewer peptide rows than before

In [ ]:
#Using these tools, it's easy to generate a lot of similar plots (like a heatmap for each different normalization type)
#Generally, the pipeline will differentiate these plots by name when they get saved
    #For heatmaps, plots are saved as 'Peptide Abundances Normalized to X heatmap.png' where X is the normalization scheme
    #However, if we save one heatmap plot and then change the replicate combination thresholds (n, std. dev.), the pipeline will overwrite the previous file since they will be named the same thing.
#In order to avoid this and to help organize different analysis directions you might pursue, the pipeline lets you switch output file directories on the fly:
exp.setOutputLocation('demo/')
#Notice the slash after the folder name: this is required to designate the folder location properly
exp.heatmap(display=False, saveFig = True)
#Now all saved files will be saved to demo/ instead of demo/output/ (which is probably not good organization)

In [ ]:
#When I've run different conditions, I create a new folder and then save to that folder
    #In order to save to a folder, it must exist before you run the program
exp.setOutputLocation('demo/output 2/')
try:
    exp.heatmap(display=False, saveFig = True)
except FileNotFoundError:
    print('ERROR: This will not work because there is no folder called output 2 inside demo yet!')
    plt.close()

In [ ]:
#Try creating a new folder and then saving to it directly


In [ ]:
#setOutputLocation can also be used to specify a prefix for all files without changing the folder by excluding the last forward slash (although I find the folder organization more useful):
exp.setOutputLocation('demo/output/this is the prefix')
exp.heatmap(display=False)

In [ ]:
#Here are some more plots to get started. Try playing around with the normalization and reference options, as well as the parameters for plots like volcano!

In [ ]:
exp.setReference('3138')
help(exp.heatmapToReference)
exp.heatmapToReference(normalization='reftime')
plt.show()

In [ ]:
help(exp.pcaToReference)
exp.pcaToReference()

In [ ]:
help(exp.pca)
exp.pca()

In [ ]:
#You may notice that the PCA plot doesn't look like the one I showed in group meeting, where 2937 and 3138 are together on the bottom right.
    #That's because we are using all 3 replicates here, whereas in the plot I showed in group meeting, I had only included the first replicate.
#Try to generate a new pca plot using only the first repicate (you may have to make a new experiment, or you can just call the function on the first replicate):


In [ ]:
help(exp.volcano)
#NOTE: this takes a long time to run because it produces a volcano for each time point and each non-reference cell line
# exp.volcano(display=True)

In [ ]:
# 3B) RUN ANALYSIS - FILES
help(exp.correlationToSelf)
#NOTE: this is a slow analysis type especially if you save files. I have saved files ahead of time in demo/output/ so you can check out what they look like
#exp.correlationToSelf()

In [ ]:
help(exp.correlationToReference)
#NOTE: this is a slow analysis type especially if you save files. I have saved files ahead of time in demo/output/ so you can check out what they look like
#exp.correlationToReference()

In [ ]:
help(exp.correlationToReferenceDiagonal)
#NOTE: this is a slow analysis type
#exp.correlationToReferenceDiagonal()

In [ ]:
#FUNCTION REFERENCE

#General functions:

#         .combineReplicates
#         .addTechnicalReplicate
#         .addPhenotypicReplicate
#         .setReference
#         .setOutputLocation

#MS analysis:

#         .heatmap
#         .heatmapToReference
#         .pca
#         .pcaToReference
#         .volcano
#         .correlationToSelf
#         .correlationToReference
#         .correlationToReferenceDiagonal

#PH analysis:

#         .groupPlot
#         .replicatePlot

#I also developed two more out-there types of MS analysis that may or may not be useful. They are not covered in this demo but are documented:

#         .log2Comparison
#         .peptidePicker

#TEMPLATE
#since we're importing these two files, we need to create a program file in the same folder as them!
import classes
import modules
#this is a module we imported from Python, so we don't have to worry about its location
import matplotlib.pyplot as plt

#replace these csv locations with the locations of your own files
locations = [
    ['filelocation/rep1,cell line 1.csv', 'filelocation/rep1,cell line 2.csv'],
    ['filelocation/rep2,cell line 1.csv', 'filelocation/rep2,cell line 2.csv'],
    ['filelocation/rep3,cell line 1.csv', 'filelocation/rep2,cell line 2.csv']]

#this is for one technical replicate for one experimental replicate
technicalReplicate = ['techrep1,cell line 1.csv', 'techrep1,cell line 2.csv']


cell_lines = ['1','2'] #strings

time_points = [0, 30, 1, 2, 5] #ints

second_time_points = [0, 30, 60, 120, 300] #ints


#	#	EXPERIMENT CREATION	#	#
exp = classes.Experiment(locations, cell_lines, time_points, second_time_points, names = ["BR1","BR2","BR3"], fileLocation = '')
#this adds to the first replicate
exp.addTechnicalReplicate(technicalReplicate,0)
exp.combineReplicates()
